# PDEfind for dataset 2 using NNs

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from aux_for_PDE_find import *
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

torch.manual_seed(0)
np.random.seed(0)

class MLP(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=16, output_dim=1, num_layers=2):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)

        self.activation = nn.GELU()

    def forward(self, inputs):
        out = self.activation(self.input_layer(inputs))
        for layer in self.hidden_layers:
            out = self.activation(layer(out))
        out = self.output_layer(out)
        return out


## Dataset preparation

In [2]:
path_train = "PDEfind_data/2.npz"
data_npz = np.load(path_train)

# Load the data from the .npz file
u = torch.from_numpy(data_npz['u']).type(torch.float32).reshape(-1,1)  # Solution or field
u.requires_grad_()
x = torch.from_numpy(data_npz['x']).type(torch.float32).reshape(-1,1)  # Spatial variable
x.requires_grad_()
t = torch.from_numpy(data_npz['t']).type(torch.float32).reshape(-1,1)   # Temporal variable
t.requires_grad_()

model = MLP()

In [3]:

batch_size = 256

training_set = DataLoader(TensorDataset(u,x,t), batch_size=batch_size, shuffle=True)


learning_rate = 0.01
epochs = 100
step_size = 50
gamma = 0.5
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

l = torch.nn.MSELoss()
freq_print = 1
for epoch in range(epochs):
    train_mse = 0.0
    for step, (u_train,x_train,t_train) in enumerate(training_set):
        inputs = torch.cat([x_train,t_train],dim=1)
        optimizer.zero_grad()
        u_pred = model(inputs)
        loss_f = l(u_pred, u_train)
        loss_f.backward()
        optimizer.step()
        train_mse += loss_f.item()
    train_mse /= len(training_set)

    scheduler.step()

    if epoch % freq_print == 0:
        model.eval()
        with torch.no_grad():
            input_nn = torch.cat([x,t],dim=1)
            u_nn = model(input_nn)
            error = (torch.norm(u-u_nn,p=2)/torch.norm(u,p=2)).item()
            print("######### Epoch:", epoch, " ######### Train Loss (%):", train_mse*100, " ######### relative L2 (%):", error*100)
        if error*100 < 1:
            break
    if error*100 < 1:
            break




######### Epoch: 0  ######### Train Loss (%): 1.27813823414568  ######### relative L2 (%): 65.80382585525513
######### Epoch: 1  ######### Train Loss (%): 0.5835614382029877  ######### relative L2 (%): 55.95027804374695
######### Epoch: 2  ######### Train Loss (%): 0.37815052140226113  ######### relative L2 (%): 37.78162598609924
######### Epoch: 3  ######### Train Loss (%): 0.11679946418648322  ######### relative L2 (%): 19.210925698280334
######### Epoch: 4  ######### Train Loss (%): 0.04223502547751223  ######### relative L2 (%): 9.880047291517258
######### Epoch: 5  ######### Train Loss (%): 0.01820981300833278  ######### relative L2 (%): 10.477690398693085
######### Epoch: 6  ######### Train Loss (%): 0.015026255764790572  ######### relative L2 (%): 11.86542734503746
######### Epoch: 7  ######### Train Loss (%): 0.009988786342858734  ######### relative L2 (%): 5.817069113254547
######### Epoch: 8  ######### Train Loss (%): 0.019796219003273566  ######### relative L2 (%): 5.9233479

In [4]:
# Save the model parameters
#torch.save(model.state_dict(), "model_params_PDE2NN.pth")
# Load the parameters into the model
model.load_state_dict(torch.load("model_params_PDE2NN.pth"))



/var/folders/2c/01bhfspx2hj56fb5ryg654lr0000gn/T/ipykernel_25349/2330747238.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_param

<All keys matched successfully>

## Build of $\Theta$

In [5]:
x = x.clone().detach()
t = t.clone().detach()

input = torch.cat([x,t],dim=1)
input.requires_grad = True
u_nn = model(input)


#compute the derivatives
grad_u = torch.autograd.grad(outputs = u_nn, inputs=input, grad_outputs=torch.ones_like(u_nn), create_graph=True, retain_graph=True)[0]
dudt = grad_u[:, 1].unsqueeze(-1)
dudx = grad_u[:, 0].unsqueeze(-1)
dudx_2 = torch.autograd.grad(dudx.sum(), input, create_graph=True, retain_graph=True)[0][:, 0].unsqueeze(-1)
dudx_3 = torch.autograd.grad(dudx_2.sum(), input, create_graph=True, retain_graph=True)[0][:, 0].unsqueeze(-1)

# Compute other terms
u2 = torch.pow(u, 2)
u3 = torch.pow(u, 3)  

# Compute mixed terms
u_times_dudx = torch.mul(u, dudx)
u2_times_dudx = torch.mul(u2, dudx)
u3_times_dudx = torch.mul(u3, dudx)

u_times_dudx_2 = torch.mul(u, dudx_2)
u2_times_dudx_2 = torch.mul(u2, dudx_2)
u3_times_dudx_2 = torch.mul(u3, dudx_2)

u_times_dudx_3 = torch.mul(u, dudx_3)
u2_times_dudx_3 = torch.mul(u2, dudx_3)
u3_times_dudx_3 = torch.mul(u3, dudx_3)

# Compute bias
bias = torch.ones_like(u_nn)


# Combine all vectors into a single matrix
Theta = torch.cat([
    bias,               # 0
    u,                  # 1
    dudx,               # 3
    dudx_2,             # 4
    dudx_3,             # 5
    u2,                 # 6
    u3,                 # 7
    u_times_dudx,       # 8
    u2_times_dudx,      # 9
    u3_times_dudx,      # 10
    u_times_dudx_2,     # 11
    u2_times_dudx_2,    # 12
    u3_times_dudx_2,    # 13
    u_times_dudx_3,     # 14
    u2_times_dudx_3,    # 15
    u3_times_dudx_3     # 16
], dim=1)

# Update Theta names
Theta_names = [
    "bias",             # 0
    "u",                # 1
    "dudx",             # 3
    "dudx_2",           # 4
    "dudx_3",           # 5
    "u2",               # 6
    "u3",               # 7
    "u_times_dudx",     # 8
    "u2_times_dudx",    # 9
    "u3_times_dudx",    # 10
    "u_times_dudx_2",   # 11
    "u2_times_dudx_2",  # 12
    "u3_times_dudx_2",  # 13
    "u_times_dudx_3",   # 14
    "u2_times_dudx_3",  # 15
    "u3_times_dudx_3"   # 16
]

## Regression and printing of the PDE

In [6]:
threshold=4
alpha = [0.1,0.01, 0.0]
Theta_= Theta.detach().numpy()
dudt_ = dudt.detach().numpy()
selected_names, regression_coef_ = myregression2(Theta_,dudt_,Theta_names,threshold=threshold,alpha=alpha)

Regression # 1
dudx  c=  -0.3833558
dudx_3  c=  -0.57645035
u_times_dudx  c=  -3.283531
Mean Squared Error: 3.2083047699416056e-05
###################### end of regression 1 ##############################

Regression # 2
dudx_3  c=  -0.78951484
u_times_dudx  c=  -4.715142
Mean Squared Error: 0.00012087886716471985
###################### end of regression 2 ##############################

Regression # 3
dudx_3  c=  -1.0055319
u_times_dudx  c=  -5.988321
Mean Squared Error: 1.768345282471273e-05
###################### end of regression 3 ##############################



In [7]:
printPDE(selected_names, regression_coef_)

dudt = -1.0055319 * dudx_3 +
       -5.988321 * u_times_dudx


# Comparison between derivatives apporximation using FDs adn NNs

In [8]:
path_train = "PDEfind_data/2.npz"
data_npz = np.load(path_train)

# Load the data from the .npz file
u_ = torch.from_numpy(data_npz['u']).type(torch.float32) # Solution or field
x_ = torch.from_numpy(data_npz['x']).type(torch.float32) # Spatial variable
t_ = torch.from_numpy(data_npz['t']).type(torch.float32)   # Temporal variable


dx = x_[1,0] - x_[0,0]
dt = t_[0,1] - t_[0,0]


dudx_fd = compute_space_derivative(u_, dx, 1).reshape(-1,1)
dudx2_fd = compute_space_derivative(u_, dx, 2).reshape(-1,1)
dudx3_fd = compute_space_derivative(u_, dx, 3).reshape(-1,1)
dudt_fd = compute_time_derivative(u_, dt).reshape(-1,1)



### Error in the function

In [9]:
torch.norm(u_nn - u.reshape(-1,1))/torch.norm(u,p=2)*100

tensor(0.9861, grad_fn=<MulBackward0>)

### Error in the first derivative in space

In [10]:
torch.norm(dudx-dudx_fd,p=2)/torch.norm(dudx_fd,p=2)*100

tensor(2.5083, grad_fn=<MulBackward0>)

### Error in the second derivative in space

In [11]:
torch.norm(dudx_2-dudx2_fd,p=2)/torch.norm(dudx2_fd,p=2)*100

tensor(5.4800, grad_fn=<MulBackward0>)

### Error in the third derivative in space

In [12]:
torch.norm(dudx_3-dudx3_fd,p=2)/torch.norm(dudx3_fd,p=2)*100

tensor(10.9865, grad_fn=<MulBackward0>)

### Error in the first derivative in time

In [13]:
torch.norm(dudt-dudt_fd,p=2)/torch.norm(dudt_fd,p=2)*100

tensor(2.4167, grad_fn=<MulBackward0>)